# How To Write A Hydro Code

Michael Zingale

There are _many_ methods for solving the equations of hydrodynamics.  We will make some choices right from the start:

  * We will consider **finite-volume methods**.  These are popular in astrophysics because they are based on the integral form of the conservative equations and properly conserve mass, momentum, and energy.
  
  * We will look at a simple 2nd order **method-of-lines** integration.  We do this for simplicity here, and will point out where things are commonly done differently.  This scheme has a much simpler spatial reconstruction and relies on an integrator (like a Runge-Kutta method) to advance in time.
  
  * We will work in 1-d.  
    
Much more in-depth details and derivations are given in my hydro notes available online: https://github.com/Open-Astrophysics-Bookshelf/numerical_exercises

For a greater variety of methods, in 2-d, see the pyro code: https://github.com/python-hydro/pyro2 (ref: [Harpole et al. JOSS](http://joss.theoj.org/papers/10.21105/joss.01265))

## Overview

We'll focus on the Euler equations.  In 1-d, these are:

\begin{align*}
  \frac{\partial \rho}{\partial t} + \frac{\partial (\rho u)}{\partial x} & = 0 \\
  \frac{\partial (\rho u)}{\partial t} + \frac{\partial (\rho u^2 + p)}{\partial x} &= 0 \\
  \frac{\partial (\rho E)}{\partial t} + \frac{\partial (u(\rho E + p))}{\partial x} &= 0 \\
  \end{align*}

This is a set of (hyperbolic) partial differential equations.  To solve these, we need to discretize the equations in both space and time.  We'll use grid-based methods (in addition to the finite-volume method, this can include finite-difference and finite-element methods).  

Our system of equations can be expressed in conservative form:
$$ \frac{\partial U}{\partial t} + \frac{\partial F(U)}{\partial x} = 0$$

In a finite-volume method, we store the state of the fluid in discrete volumes in space, and we can refer to this discretized state with an index.  To see this, we integrate the conservative law system in space over a volume $[x_{i-1/2},x_{i+1/2}]$:
$$\frac{\partial \langle U\rangle_i}{\partial t} = - \frac{F_{i+1/2} - F_{i-1/2}}{\Delta x}$$

This is the form of the equations we will solve.  Here, $\langle U\rangle_i$ represents the average state of the fluid in a volume:
$$\langle U\rangle_i = \frac{1}{\Delta x} \int_{x_{i-1/2}}^{x_{i+1/2}} U(x) dx$$

The state on the grid represents an instance in time.  We evolve the state by computing the fluxes through the volumes.  These fluxes tell us how much the state changes in each volume over some small timestep, $\Delta t$.  

Our code will have the following structure:

  * Create our numerical grid
  
  * Set the initial conditions
  
  * Main timestep evolution loop
  
    * Compute the timestep
    
    * Time-integation loop (depends on the number of stages in the integrator)
  
        * Reconstruct the state to interfaces
    
        * Solve Riemann problem to find the fluxes through the interface
    
        * Do a conservative update of the state to the stage
    
    * Output

## Grid

In [3]:
class FVGrid:

    def __init__(self, nx, ng, xmin=0.0, xmax=1.0):

        self.xmin = xmin
        self.xmax = xmax
        self.ng = ng
        self.nx = nx

        # python is zero-based.  Make easy intergers to know where the
        # real data lives
        self.ilo = ng
        self.ihi = ng+nx-1

        # physical coords -- cell-centered
        self.dx = (xmax - xmin)/(nx)
        self.x = xmin + (np.arange(nx+2*ng)-ng+0.5)*self.dx

    def scratch_array(self, nc=1):
        """ return a scratch array dimensioned for our grid """
        if nc == 1:
            return np.zeros((self.nx+2*self.ng), dtype=np.float64)
        else:
            return np.zeros((self.nx+2*self.ng, nc), dtype=np.float64)

    def fill_BCs(self, atmp):
        """ fill all ghost cells with zero-gradient boundary conditions """

        try:
            nc = atmp.shape[1]
        except:
            nc = 1

        if nc == 1:
            atmp[0:self.ilo] = atmp[self.ilo]
            atmp[self.ihi+1:] = atmp[self.ihi]
        else:
            for n in range(nc):
                atmp[0:self.ilo, n] = atmp[self.ilo, n]
                atmp[self.ihi+1:, n] = atmp[self.ihi, n]

## Reconstruction

In [5]:
def states(grid, U):
    
    # convert to primitive
    q = grid.scratch_array(nc=NVAR)

    q[:, QRHO] = U[:, URHO]
    q[:, QU] = U[:, UMX]/U[:, URHO]
    q[:, QP] = (U[:, UENER] - 0.5*q[:, QRHO]*q[:, QU]**2)*(gamma - 1.0)    

    # construct the slopes
    dq = grid.scratch_array(nc=NVAR)

    for n in range(NVAR):
        # unlimited centered slopes
        dq[ib:ie+1,n] = 0.5*(q[ib+1:ie+2,n] - q[ib-1:ie,n])

    # now make the states
    q_l = grid.scratch_array(nc=NVAR)
    q_l[grid.ilo:grid.ihi+2, :] = q[grid.ilo-1:grid.ihi+1, :] + 0.5*dq[grid.ilo-1:grid.ihi+1, :]

    q_r = grid.scratch_array(nc=NVAR)
    q_r[grid.ilo:grid.ihi+2, :] = q[grid.ilo:grid.ihi+2, :] - 0.5*dq[self.gr.ilo:self.gr.ihi+2, :]

## Riemann problem

## Conservative update

## Main driver

## Example: Sod's problem